[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anonym-gm/asplenia_EHR_embedding/blob/main/embedding_INA.ipynb)

## Libraries

In [1]:
import torch
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore')
import numpy as np
from sklearn.metrics import (
    matthews_corrcoef, confusion_matrix, accuracy_score, roc_auc_score,
    precision_score, recall_score, f1_score
)
import lightgbm as lgb
import seaborn as sns
from tqdm.notebook import tqdm
import sys, os
import random
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"       # to force BERT determinsm

def set_seed(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # per multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)


## Download data

In [2]:
!git clone https://github.com/anonym-gm/asplenia_EHR_embedding.git
%cd asplenia_EHR_embedding

Cloning into 'asplenia_EHR_embedding'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 122 (delta 53), reused 98 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (122/122), 1.32 MiB | 4.53 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/Users/maurizio/Library/CloudStorage/GoogleDrive-maurizio.giordano@icar.cnr.it/Il mio Drive/asplenia_EHR_embedding/asplenia_EHR_embedding


## Load dataset

In [3]:
datapath = 'data'
lang = 'EN'
df = pd.read_json(os.path.join(datapath, f"dataset_{lang}.json")).set_index("id")
groupname = "ALL"
selected_ids = pd.read_csv(os.path.join(datapath, f"{groupname}_index.csv"), header=None).values.ravel()
num_epochs = 10
batch_size = 32
embedding_dim = 64
hidden_dim = 128
seed = 42
enable_plot = True
methods=['BINARY', 'DOME', 'LSTM', 'tLSTM', 'GRU', 'GRU-D', 'BEHRT', 'RETAIN', 'Dipole']
dataset = df.loc[selected_ids]
dataset

,is_splenectomized?,qt_vax_prophylaxis,days_after,days_before,qt_vax_post_prophylaxis,tsh,dosage_num,qt_vax_totali,smoking,dyslipidemia,...,splenectomy_indication,splenectomy_method,primary_disease,prophylaxis_vax_hib,prophylaxis_vax_flu,prophylaxis_vax_meningo,prophylaxis_vax_pneumo,splenectomy_response,gender,events
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0.0,0.0,5,NaN,-1000.0,4,0.0,0.0,...,NaN,0.0,2,0,0,0,0,NaN,0,"[{'type': 'therapy', 'event': 'antiplatelet ag..."
4,1,6,0.0,0.0,2,NaN,-1000.0,8,0.0,0.0,...,NaN,1.0,0,1,0,1,1,NaN,1,"[{'type': 'vaccination', 'event': 'Hib', 'date..."
8,1,0,0.0,0.0,1,NaN,-1000.0,1,0.0,0.0,...,NaN,0.0,7,0,0,0,0,NaN,1,"[{'type': 'surgical operation', 'event': 'sple..."
9,1,3,0.0,0.0,1,NaN,-1000.0,4,0.0,0.0,...,NaN,0.0,2,1,0,1,1,NaN,0,"[{'type': 'vaccination', 'event': 'Hib', 'date..."
10,1,3,0.0,0.0,0,NaN,-1000.0,3,0.0,0.0,...,NaN,0.0,0,1,0,1,1,NaN,0,"[{'type': 'therapy', 'event': 'anticoagulant',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,1,11,0.0,0.0,0,NaN,-1000.0,11,0.0,0.0,...,NaN,NaN,0,1,0,1,1,NaN,1,"[{'type': 'vaccination', 'event': 'Hib', 'date..."
1949,1,9,0.0,0.0,0,NaN,-1000.0,9,0.0,0.0,...,NaN,NaN,0,1,0,1,1,NaN,1,"[{'type': 'vaccination', 'event': 'PCV13', 'da..."
1950,1,9,0.0,0.0,0,NaN,-1000.0,9,0.0,0.0,...,NaN,NaN,0,1,0,1,1,NaN,1,"[{'type': 'vaccination', 'event': 'Hib', 'date..."


## Get sequences and static data

In [ ]:
from src.utils import count_events_by_type
ev_fields = ['event']
target_type = "infection"
events_field, date_field, type_field = 'events', 'date', 'type'
removeevents = ['followup', 'platelet_change']
attributes = pd.read_csv(os.path.join(datapath, "attributes_EN.csv"), header=None, comment = '#', index_col=0).index.to_list()
event_sequences_no_trunc_with_type = { id: [(" ".join([event[field] for field in ev_fields]), event[date_field], event[type_field]) for event in events] for (id,events) in dataset[[events_field]].to_dict()[events_field].items()} 
cnt_df = count_events_by_type(event_sequences_no_trunc_with_type)

## Load targets

In [5]:
from src.utils import group_events_by_visit
# get sequence of event names
event_sequences_no_trunc_with_type = { id: [(" ".join([event[field] for field in ev_fields]), event[date_field], event[type_field]) for event in events if event[type_field] not in removeevents] for (id,events) in dataset[[events_field]].to_dict()[events_field].items()} 
event_sequences_no_trunc = { id: [(" ".join([event[field] for field in ev_fields]), event[date_field]) for event in events if event[type_field] not in removeevents] for (id,events) in dataset[[events_field]].to_dict()[events_field].items()} 
visit_sequences_no_trunc = group_events_by_visit(event_sequences_no_trunc)
print(f"n.{len(event_sequences_no_trunc)} Patients")


n.1067 Patients


In [6]:
from src.utils import count_events_by_type
count_events_by_type(event_sequences_no_trunc_with_type)

,cardinality,n. instances,set
type,,,
therapy,34,3831,"{antipsychotic, vasodilator, antiviral, antihy..."
surgical operation,13,1129,"{kidney transplant, adenoidectomy, cholecystec..."
vaccination,8,4589,"{Hib, meningococcus_C, meningococcus_B, PCV7, ..."
infection,37,346,"{bacterial/viral infection of the ear, sexuall..."
comorbidity,15,647,"{nervous system diseases, cardiovascular disea..."
thrombosis,4,160,"{thromboembolism, venous thrombosis, arterial/..."


## Truncate sequences on 1st target

In [7]:
from src.utils import truncate_events_on1st_infection
event_sequences, y_df = truncate_events_on1st_infection(event_sequences_no_trunc_with_type, field_value=target_type)
visit_sequences = group_events_by_visit(event_sequences)

Truncating:   0%|          | 0/1067 [00:00<?, ?it/s]

## Configurations

In [8]:
from src.embedding import LSTMembedder, BEHRTembedder, COUNTEREmbedder, TimeAwareLSTMEmbedder, DipoleEmbedder 
from src.embedding import StaticEmbedder, RETAINembedder, DOMEEmbedder, BINARYEmbedder, GRUEmbedder, GRUEDembedder
# Definisci la lista dei "parametri" degli embedder
# === LSTM vcabulary ===
vocab = set()
for patient_events in event_sequences_no_trunc.values():
    for event,date in patient_events:
        vocab.update([event] if isinstance(event, str) else event)
word_to_idx = {word: idx for idx, word in enumerate(sorted(vocab))}  # for LSTM, RETAIN, etc
# === BEHRT vocabulary ===
code2id = {"[PAD]": 0, "[CLS]": 1, "[SEP]": 2}
idx = 3
for pid, visits in visit_sequences_no_trunc.items():
    for events, date in visits:
        for event in events:
            if event not in code2id:
                code2id[event] = idx
                idx += 1

labels = y_df.to_dict()['target']
embedder_configs =  { 
        "LSTM" : 
        {   "func": LSTMembedder,
            "kwargs": {
                "sequences": event_sequences,
                "labels": labels,
                "word_to_idx": word_to_idx,
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "embed_size": embedding_dim,
                "hidden_size": hidden_dim,
                "enable_plot": enable_plot
            }
        },
      "tLSTM" : 
        {   "func": TimeAwareLSTMEmbedder,
            "kwargs": {
                "sequences": event_sequences,
                "labels": labels,
                "word_to_idx": word_to_idx,
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "embed_size": embedding_dim,
                "hidden_size": hidden_dim,
                "enable_plot": enable_plot
            }
        },
      "RETAIN" : 
        {   "func": RETAINembedder,
            "kwargs": {
                "sequences": visit_sequences,
                "labels": labels,
                "word_to_idx": code2id,
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "embed_size": embedding_dim,
                "hidden_size": hidden_dim,
                "enable_plot": enable_plot
            }
        },
      "BEHRT" : 
        {   "func": BEHRTembedder,
            "kwargs": {
                "sequences": visit_sequences,
                "labels": labels,
                "word_to_idx": code2id,
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "embed_size": embedding_dim,
                "hidden_size": hidden_dim,
                "enable_plot": enable_plot
            }
        },
      "Dipole" : 
        {   "func": DipoleEmbedder,
            "kwargs": {
                "sequences": event_sequences,
                "labels": labels,
                "word_to_idx": code2id,
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "embed_size": embedding_dim,
                "hidden_size": hidden_dim,
                "enable_plot": enable_plot
            }
        },
        "GRU" : 
        {   "func": GRUEmbedder,
            "kwargs": {
                "sequences": event_sequences,
                "labels": labels,
                "word_to_idx": word_to_idx,
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "embed_size": embedding_dim,
                "hidden_size": hidden_dim,
                "enable_plot": enable_plot
            }
        },       "GRU-D" : 
        {   "func": GRUEDembedder,
            "kwargs": {
                "sequences": visit_sequences,
                "labels": labels,
                "word_to_idx": code2id,
                "num_epochs": num_epochs,
                "batch_size": batch_size,
                "embed_size": embedding_dim,
                "hidden_size": hidden_dim,
                "enable_plot": enable_plot
            }
        },
       "STATIC" : 
        {   "func": StaticEmbedder,
            "kwargs": {
                "df": dataset,
                "include_attributes": attributes,
                "enable_plot": enable_plot
            }
        },
      "DOME" :
        {
            "func": DOMEEmbedder,
            "kwargs": {
                 "sequences": event_sequences_no_trunc,
                 "targets": targets,
                 "df": dataset,
                 "enable_plot": enable_plot
            }
        },
        "BINARY": 
        {
            "func": BINARYEmbedder,
            "kwargs": {
                "sequences": event_sequences,
                "targets": targets,
                "vocab": vocab,
                "enable_plot": enable_plot
            }
        },
        "COUNTER": 
        {
            "func": COUNTEREmbedder,
            "kwargs": {
                "sequences": event_sequences,
                "targets": targets,
                "vocab": vocab,
                "enable_plot": enable_plot
            }
        }
    }


## Validate

In [ ]:
from src.cv_folding import lgbm_cv
set_seed(seed)
results_df, model, _,_,_,_ = lgbm_cv(embedder_configs, y_df, selected_ids, methods=["STATIC"])
for m in methods:
    set_seed(seed)
    results_tmp, model, _,_,_,_ = lgbm_cv(embedder_configs, y_df, selected_ids, methods=["STATIC", m])
    results_df = pd.concat([results_df, results_tmp],axis=0)
results_df

## Latex tables 5-6 in the paper

In [ ]:
from src.utils import df_to_latex_bold
print(df_to_latex_bold(results_df))

\begin{tabular}{llllllll}
\toprule
 & AUC & F1 & Prec & Recall & MCC & Acc & CM \\
\midrule
STATIC & 0.856$\pm$0.038 & 0.642$\pm$0.106 & 0.785$\pm$0.184 & 0.627$\pm$0.207 & 0.569$\pm$0.068 & 0.818$\pm$0.036 & [[283  34]
 [ 46  77]] \\
STATIC+BINARY & 0.883$\pm$0.049 & 0.762$\pm$0.078 & 0.872$\pm$0.031 & 0.691$\pm$0.133 & 0.702$\pm$0.074 & 0.884$\pm$0.026 & [[304  13]
 [ 38  85]] \\
STATIC+DOME & \textbf{0.896$\pm$0.042} & 0.781$\pm$0.075 & 0.772$\pm$0.123 & \textbf{0.806$\pm$0.067} & 0.698$\pm$0.106 & 0.870$\pm$0.055 & [[284  33]
 [ 24  99]] \\
STATIC+LSTM & 0.850$\pm$0.034 & 0.704$\pm$0.056 & 0.757$\pm$0.056 & 0.666$\pm$0.094 & 0.606$\pm$0.070 & 0.845$\pm$0.026 & [[290  27]
 [ 41  82]] \\
STATIC+tLSTM & 0.841$\pm$0.054 & 0.697$\pm$0.126 & 0.767$\pm$0.073 & 0.684$\pm$0.200 & 0.618$\pm$0.112 & 0.848$\pm$0.038 & [[289  28]
 [ 39  84]] \\
STATIC+GRU & 0.819$\pm$0.063 & 0.696$\pm$0.056 & 0.730$\pm$0.075 & 0.666$\pm$0.050 & 0.586$\pm$0.080 & 0.836$\pm$0.033 & [[286  31]
 [ 41  82]] \\
STATI

In [ ]:
outpath = "output"
results_df.to_csv(os.path.join(outpath, f"results_ina_{groupname}.csv"))

## SHAP analysis

In [ ]:
from src.cv_folding import lgbm_cv
results_df, model, X_df, Xv_df, y_train, y_test = lgbm_cv(embedder_configs, y_df, selected_ids, methods=["STATIC", "BINARY"])
results_df

In [ ]:
import shap
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(Xv_df)
shap.plots.violin(shap_values, features=Xv_df, feature_names=Xv_df.columns,max_display=15)


## Heatmaps

In [ ]:
from src.cv_folding import lgbm_cv
results_df, model, X_df, Xv_df, y_train, y_test = lgbm_cv(embedder_configs, y_df, selected_ids, methods=["COUNTER"])
results_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
import matplotlib as mpl

mpl.rcParams['font.size'] = 20
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['axes.labelsize'] = 22
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 14.5
mpl.rcParams['legend.fontsize'] = 16
mpl.rcParams['legend.title_fontsize'] = 16
X_df_all = pd.concat([X_df, Xv_df], axis=0)
X_df_all['target'] = y_df.loc[X_df_all.index, 'target']
X_sorted = X_df_all.sort_values('target', ascending=False)
targets_sorted = X_sorted['target'].values
X_sorted = X_sorted.drop(columns='target')
X_norm = X_sorted.copy()
X_norm_t = X_norm.T
scaler = MinMaxScaler()
X_norm_t.iloc[:, :] = scaler.fit_transform(X_norm_t)
X_filtered = X_norm_t[~(X_norm_t == 0).all(axis=1)]
row_colors = ['yellow' if y_df.loc[idx, 'target'] == 1 else 'orange' for idx in X_filtered.columns]
legend_elements = [Patch(facecolor='yellow', edgecolor='k', label='Class 1 = With Infectious Events'),
                   Patch(facecolor='orange', edgecolor='k', label='Class 0 = Without Infectious Events')]

g = sns.clustermap(
    X_filtered,
    cmap='RdBu_r',
    cbar_pos=(0.03, 0.1, 0.02, 0.18),
    figsize=(16, 14),
    col_colors=row_colors,
    xticklabels=False,
    yticklabels=True,
    col_cluster=False,
    row_cluster=True,
    metric='euclidean',
    method='ward'
)
# Compute rows with target 1
n_positive = sum(y_df.loc[X_filtered.columns, 'target'] == 1)

g.ax_heatmap.axvline(x=n_positive, color='white', linestyle='-', linewidth=2)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xticklabels())
plt.gcf().legend(handles=legend_elements, title='Target class',
                 loc='upper center', bbox_to_anchor=(0.4, 0.5))
plt.show()